<a href="https://colab.research.google.com/github/jsstar522/hunkim_ML/blob/master/03_NeuralNet_ReLU/04_MNIST_with_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MNIST with Deep Learning

이제 Softmax classification로 MNIST 예측을 했던 문제를 Deep Learning을 통해 해결해보겠습니다.

## Softmax
먼저 Softmax를 통한 정확도를 뽑아보겠습니다.

In [15]:
import tensorflow as tf
import numpy
## MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("Data/MNIST_data/", one_hot=True)

## 28x28
X = tf.placeholder(tf.float32, shape=[None, 784])
## 0~9
Y = tf.placeholder(tf.float32, shape=[None, 10])
W = tf.Variable(tf.random_normal([784, 10]))
b = tf.Variable(tf.random_normal([10]))

## hypothesis
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

## cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))

## minimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

## initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## accuracy
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, dtype=tf.float32))

## set epochs & batch size
training_epochs = 10
batch_size = 100
for epoch in range(training_epochs):
    avg_cost = 0
    iteration = int(mnist.train.num_examples/batch_size)
    ## 전체 데이터에서 batch_size를 나눈 iteration
    for i in range(iteration):
        ## training dataset
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c/iteration
        
    print("Epoch:", "%04d" %(epoch + 1), "Cost: ", "{:.9f}" .format(avg_cost))

## testing dataset
print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))
    

Extracting Data/MNIST_data/train-images-idx3-ubyte.gz
Extracting Data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting Data/MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 Cost:  4.655570388
Epoch: 0002 Cost:  1.583994455
Epoch: 0003 Cost:  1.069022316
Epoch: 0004 Cost:  0.854501970
Epoch: 0005 Cost:  0.734014441
Epoch: 0006 Cost:  0.656466654
Epoch: 0007 Cost:  0.600799085
Epoch: 0008 Cost:  0.558425213
Epoch: 0009 Cost:  0.525775306
Epoch: 0010 Cost:  0.498680057
Accuracy:  0.886


## Nueral Network

이제 일반적으로 사용하는 multi-layer 형태의 Neural Network를 사용해보겠습니다.

In [14]:
import tensorflow as tf
import numpy
## MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("Data/MNIST_data/", one_hot=True)

## 28x28
X = tf.placeholder(tf.float32, shape=[None, 784])
## 0~9
Y = tf.placeholder(tf.float32, shape=[None, 10])

## layer 1
W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

## layer 2
W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

## layer 3
W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

## cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))

## minimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

## initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## accuracy
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, dtype=tf.float32))

## set epochs & batch size
training_epochs = 10
batch_size = 100
for epoch in range(training_epochs):
    avg_cost = 0
    iteration = int(mnist.train.num_examples/batch_size)
    ## 전체 데이터에서 batch_size를 나눈 iteration
    for i in range(iteration):
        ## training dataset
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c/iteration
        
    print("Epoch:", "%04d" %(epoch + 1), "Cost: ", "{:.9f}" .format(avg_cost))

## testing dataset
print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Extracting Data/MNIST_data/train-images-idx3-ubyte.gz
Extracting Data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting Data/MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 Cost:  195.752253359
Epoch: 0002 Cost:  44.764822620
Epoch: 0003 Cost:  27.828198097
Epoch: 0004 Cost:  19.627305823
Epoch: 0005 Cost:  14.092954263
Epoch: 0006 Cost:  10.608012293
Epoch: 0007 Cost:  8.096181757
Epoch: 0008 Cost:  6.040426032
Epoch: 0009 Cost:  4.424856464
Epoch: 0010 Cost:  3.443676003
Accuracy:  0.9429


## Xavier initialization

이번에는 Weight 초기화를 통해 정확도를 높여보겠습니다. Weight 값을 설정할 때, `tf.contrib.layers.xavier_initializer()`를 사용하면 `Xavier 초기화`를 사용할 수 있습니다.

In [13]:
import tensorflow as tf
import numpy
## MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("Data/MNIST_data/", one_hot=True)

tf.reset_default_graph()

## 28x28
X = tf.placeholder(tf.float32, shape=[None, 784])
## 0~9
Y = tf.placeholder(tf.float32, shape=[None, 10])

## layer 1
W1 = tf.get_variable("W1", shape=[784, 256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

## layer 2
W2 = tf.get_variable("W2", shape=[256, 256], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

## layer 3
W3 = tf.get_variable("W3", shape=[256, 10], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

## cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))

## minimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

## initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## accuracy
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, dtype=tf.float32))

## set epochs & batch size
training_epochs = 10
batch_size = 100
for epoch in range(training_epochs):
    avg_cost = 0
    iteration = int(mnist.train.num_examples/batch_size)
    ## 전체 데이터에서 batch_size를 나눈 iteration
    for i in range(iteration):
        ## training dataset
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c/iteration
        
    print("Epoch:", "%04d" %(epoch + 1), "Cost: ", "{:.9f}" .format(avg_cost))

## testing dataset
print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Extracting Data/MNIST_data/train-images-idx3-ubyte.gz
Extracting Data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting Data/MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 Cost:  0.310727648
Epoch: 0002 Cost:  0.113794084
Epoch: 0003 Cost:  0.076681319
Epoch: 0004 Cost:  0.055887896
Epoch: 0005 Cost:  0.040908583
Epoch: 0006 Cost:  0.030830162
Epoch: 0007 Cost:  0.023312608
Epoch: 0008 Cost:  0.018209115
Epoch: 0009 Cost:  0.017060772
Epoch: 0010 Cost:  0.014571132
Accuracy:  0.9808


## Dropout

먼저 layer를 더 깊게 만들고 node를 더 많이 생성(wide)해서 학습을 시켜보겠습니다. 

In [12]:
import tensorflow as tf
import numpy
## MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("Data/MNIST_data/", one_hot=True)

tf.reset_default_graph()

## 28x28
X = tf.placeholder(tf.float32, shape=[None, 784])
## 0~9
Y = tf.placeholder(tf.float32, shape=[None, 10])

## layer 1
W1 = tf.get_variable("W1", shape=[784, 512], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

## layer 2
W2 = tf.get_variable("W2", shape=[512, 512], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

## layer 3
W3 = tf.get_variable("W3", shape=[512, 512], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

## layer 4
W4 = tf.get_variable("W4", shape=[512, 512], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

## layer 5
W5 = tf.get_variable("W5", shape=[512, 10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

## cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))

## minimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

## initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## accuracy
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, dtype=tf.float32))

## set epochs & batch size
training_epochs = 10
batch_size = 100
for epoch in range(training_epochs):
    avg_cost = 0
    iteration = int(mnist.train.num_examples/batch_size)
    ## 전체 데이터에서 batch_size를 나눈 iteration
    for i in range(iteration):
        ## training dataset
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys})
        avg_cost += c/iteration
        
    print("Epoch:", "%04d" %(epoch + 1), "Cost: ", "{:.9f}" .format(avg_cost))

## testing dataset
print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Extracting Data/MNIST_data/train-images-idx3-ubyte.gz
Extracting Data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting Data/MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 Cost:  0.312850165
Epoch: 0002 Cost:  0.103657596
Epoch: 0003 Cost:  0.070952105
Epoch: 0004 Cost:  0.053882102
Epoch: 0005 Cost:  0.041925784
Epoch: 0006 Cost:  0.036304727
Epoch: 0007 Cost:  0.027852293
Epoch: 0008 Cost:  0.026560210
Epoch: 0009 Cost:  0.025539212
Epoch: 0010 Cost:  0.021148856
Accuracy:  0.9767


더 Deep하고 Wide하게 학습을 했음에도 불구하고 결과가 비슷하거나 더 나빠지는 것을 확인할 수 있습니다. 이렇게 Deep하고 Wide 해지면 Overfitting이 발생했을 확률이 굉장히 높습니다. 변수가 굉장히 많기 때문에 testing data에 제대로 적용시키지 못하는 것이죠. 이 문제를 해결하기 위해 `Dropout`을 사용해보겠습니다. Dropout은 임의의 node를 끄는 방법입니다. layer마다 dropout 함수를 적용시켜줍니다. 이 때 몇퍼센트의 node를 사용할 것인지 정하는 것이 dropout 함수의 두번째 인자, keep_prob에 들어가게 되는데 **학습할 때는 대부분 70%, 실제 사용시에는 100%를 사용해야하므로 사전에 keep_prob를 placeholder로 지정(변수)한 뒤 학습과 실전에서 feed_dict에서 값을 넣어 사용**하면 됩니다.

In [16]:
import tensorflow as tf
import numpy
## MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("Data/MNIST_data/", one_hot=True)
tf.reset_default_graph()

## dropout probabilty
keep_prob = tf.placeholder(tf.float32)

## 28x28
X = tf.placeholder(tf.float32, shape=[None, 784])
## 0~9
Y = tf.placeholder(tf.float32, shape=[None, 10])

## layer 1
W1 = tf.get_variable("W1", shape=[784, 512], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([512]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
## keep_prob
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

## layer 2
W2 = tf.get_variable("W2", shape=[512, 512], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

## layer 3
W3 = tf.get_variable("W3", shape=[512, 512], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)

## layer 4
W4 = tf.get_variable("W4", shape=[512, 512], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

## layer 5
W5 = tf.get_variable("W5", shape=[512, 10], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

## cost
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))

## minimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

## initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## accuracy
is_correct = tf.equal(tf.arg_max(hypothesis, 1), tf.arg_max(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, dtype=tf.float32))

## set epochs & batch size
training_epochs = 10
batch_size = 100
for epoch in range(training_epochs):
    avg_cost = 0
    iteration = int(mnist.train.num_examples/batch_size)
    ## 전체 데이터에서 batch_size를 나눈 iteration
    for i in range(iteration):
        ## training dataset
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        avg_cost += c/iteration
        
    print("Epoch:", "%04d" %(epoch + 1), "Cost: ", "{:.9f}" .format(avg_cost))

## testing dataset
print("Accuracy: ", accuracy.eval(session=sess, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

Extracting Data/MNIST_data/train-images-idx3-ubyte.gz
Extracting Data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting Data/MNIST_data/t10k-labels-idx1-ubyte.gz
Epoch: 0001 Cost:  0.456425124
Epoch: 0002 Cost:  0.172553552
Epoch: 0003 Cost:  0.128965483
Epoch: 0004 Cost:  0.112313464
Epoch: 0005 Cost:  0.094356168
Epoch: 0006 Cost:  0.084927326
Epoch: 0007 Cost:  0.077305277
Epoch: 0008 Cost:  0.069215620
Epoch: 0009 Cost:  0.064746225
Epoch: 0010 Cost:  0.058119157
Accuracy:  0.9831
